In [2]:
#Import libraries
import pandas as pd
import nltk

In [3]:
# Read training data csv file
data_train=pd.read_csv('train_2kmZucJ.csv')

In [4]:
# See top 5 records of the data
data_train.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [5]:
#List of ids of training data
ids_train=data_train['id'].tolist()

In [6]:
#Read test data csv file
data_test=pd.read_csv('test_oJQbWVk.csv')

In [7]:
#See top 5 records of data
data_test.head()

,id,tweet
0,7921,I hate the new #iphone upgrade. Won't let me d...
1,7922,currently shitting my fucking pants. #apple #i...
2,7923,"I'd like to puts some CD-ROMS on my iPad, is t..."
3,7924,My ipod is officially dead. I lost all my pict...
4,7925,Been fighting iTunes all night! I only want th...


In [8]:
#Combine test and training data
comb_data=data_train.append(data_test,ignore_index=True)

C:\Users\HP\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


# Data Cleaning

In [9]:
from nltk.stem.porter import *
stemmer=PorterStemmer()

In [10]:
#Data Cleaning function
def data_clean(x):
    x_remove_pattern=re.sub("@[\w]*"," ",x)
    x_remove_pattern=re.sub("$&@*#"," ",x)
    x_replace_words=x_remove_pattern.replace("[^a-zA-Z0-9#]"," ")
    x_new=' '.join([w for w in x_replace_words.split(' ') if len(w)>=3])
    return x_replace_words

In [11]:
#Clean all the tweets
comb_data['tidy_tweet']=comb_data['tweet'].apply(lambda x:data_clean(x))

# Data Visualization 

In [12]:
# Import libraries for data visualization
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# All words WordCloud

In [15]:
# Generate word cloud of all the words
allwords=' '.join([text for text in comb_data['tidy_tweet']])
wordcloud=WordCloud().generate(allwords)
plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()

<Figure size 720x504 with 0 Axes>

<Figure size 720x504 with 0 Axes>

# Positive Words WordCloud

In [16]:
# Generate word cloud of positive words
positivewords=' '.join([text for text in comb_data['tidy_tweet'][comb_data['label']==0]])
wordcloud=WordCloud().generate(positivewords)
plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()

<Figure size 720x504 with 0 Axes>

<Figure size 720x504 with 0 Axes>

# Negative Words WordCloud

In [17]:
#Generate negative word clouds
negativewords=' '.join([text for text in comb_data['tidy_tweet'][comb_data['label']==1]])
wordcloud=WordCloud().generate(negativewords)
plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()

<Figure size 720x504 with 0 Axes>

<Figure size 720x504 with 0 Axes>

# Feature Extraction

In [370]:
#Tf-IDF feature extraction
from sklearn.feature_extraction.text import TfidfVectorizer
feature_extractor=TfidfVectorizer(max_df=0.9, min_df=2, max_features=1500, stop_words='english')

In [371]:
#Split train data and test data based on ids
train_data=comb_data[comb_data['id'].isin(ids_train)]
test_data=comb_data[~(comb_data['id'].isin(ids_train))]

In [372]:
#feature extraction of train and test data
features_train=feature_extractor.fit_transform(train_data['tidy_tweet'])
features_test=feature_extractor.transform(test_data['tidy_tweet'])

In [373]:
#Split data into training and validation
from sklearn.model_selection import train_test_split
X_train,X_val_1,y_train,y_val_1=train_test_split(features_train,train_data['label'],random_state=42,test_size=0.2,stratify=train_data['label'])

In [374]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold,StratifiedKFold

In [375]:
#Build Random Forest classification model
"""from sklearn.ensemble import RandomForestClassifier
kcv=StratifiedKFold(n_splits=5,random_state=42,shuffle=False)
classifier=RandomForestClassifier(random_state=42,min_samples_leaf=10,n_estimators=2000)
scores=[]
for train_index,test_index in kcv.split(bow_train,train_data['label']):
    X_train,X_val,y_train,y_val=features_train[train_index],features_train[test_index],train_data['label'][train_index],train_data['label'][test_index]
    classifier.fit(X_train,y_train)
    scores.append(classifier.score(X_val,y_val))
scores"""

"from sklearn.ensemble import RandomForestClassifier\nkcv=StratifiedKFold(n_splits=5,random_state=42,shuffle=False)\nclassifier=RandomForestClassifier(random_state=42,min_samples_leaf=10,n_estimators=2000)\nscores=[]\nfor train_index,test_index in kcv.split(bow_train,train_data['label']):\n    X_train,X_val,y_train,y_val=features_train[train_index],features_train[test_index],train_data['label'][train_index],train_data['label'][test_index]\n    classifier.fit(X_train,y_train)\n    scores.append(classifier.score(X_val,y_val))\nscores"

In [376]:
#XGBoost Classifier
from xgboost import XGBClassifier
kcv=StratifiedKFold(n_splits=5,random_state=42,shuffle=False)
classifier=XGBClassifier(random_state=42,n_estimators=1000)
classifier.fit(X_train,y_train)
scores=[]
for train_index,test_index in kcv.split(bow_train,train_data['label']):
    X_train,X_val,y_train,y_val=features_train[train_index],features_train[test_index],train_data['label'][train_index],train_data['label'][test_index]
    classifier.fit(X_train,y_train)
    scores.append(classifier.score(X_val,y_val))
scores

[0.8883280757097792,
 0.8787878787878788,
 0.8882575757575758,
 0.8800505050505051,
 0.8856601389766267]

In [377]:
from sklearn.metrics import f1_score

In [378]:
#Get the  prediction of validation data
import numpy as np
prediction=classifier.predict(X_val_1)
f1_score(y_val_1,prediction)

0.8807785888077858

In [379]:
#Get prediction on test data and create submission file
test_data['label']=(classifier.predict(features_test))
submission=test_data[['id','label']]
submission.to_csv('submission.csv',index=False)

C:\Users\HP\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
